# 1. MLflow Basic Example with Sklearn

### Train and save model to MLflow

In [10]:
import numpy as np
import mlflow as mlf


from mlflow.models.signature import infer_signature
import mlflow.sklearn
import pandas as pd
import pymysql
from sklearn.metrics import precision_score, recall_score;
from sklearn.model_selection import cross_val_predict;
from xgboost import XGBClassifier;
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import numpy as np
import pickle

# Let's create some data
def limpiado(data, columnas_one_hot):
    data = pd.get_dummies(data, columns=columnas_one_hot)

    data['id Sleep Disorder'] = data['id Sleep Disorder'] - 1

    #Parte del balanceo de dat
    X = data.drop("id Sleep Disorder", axis=1)  
    y = data["id Sleep Disorder"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30) 

    return X_train, y_train, X_test, y_test


# Establecer la conexión a la base de datos
connection = pymysql.connect(
    host='172.29.192.1',
    user='jose',
    password='abc1234',
    db='sleep'
)

# Consultar los datos de una tabla específica
query = """
SELECT f.*, d1.Gender, d2.Occupation, d3.`BMI Category`
FROM `facts` f
JOIN `gender` d1 ON f.`id Gender` = d1.`id Gender`
JOIN `occupation` d2 ON f.`id Occupation` = d2.`id Occupation`
JOIN `bmi` d3 ON f.`id BMI Category` = d3.`id BMI Category`
"""

df = pd.read_sql_query(query, connection)
df.set_index('Person ID', inplace=True)
cols = ['id Gender', 'id Occupation', 'id BMI Category']
df.drop(cols, axis=1, inplace=True)

connection.close()

#LLamado a la función
columnas_one_hot = ["BMI Category","Occupation","Gender"]
X_train, y_train, X_test, y_test = limpiado(df, columnas_one_hot)

params = {'n_estimators': 100, 'random_State': 42}
xgb_clf = XGBClassifier(params)
xgb_clf.fit(X_train, y_train)

y_train_pred_xgb = cross_val_predict(xgb_clf, X_train, y_train, cv=5)

precision = precision_score(y_train, y_train_pred_xgb, average=None)
recall = recall_score(y_train, y_train_pred_xgb, average=None)

print("Precisión:\t", np.round(precision, 2), np.mean(precision))
print("Recall:\t\t", np.round(recall, 2), np.mean(recall))

with mlf.start_run() as run:
    mlf.log_metric("precision", np.mean(precision))
    mlf.log_metric("recall", np.mean(recall))
    
    mlf.log_params(params)
    predictions = xgb_clf.predict(X_train)

    signature = infer_signature(X_train, predictions)

    artifact_path = "model"
    mlf.sklearn.log_model(xgb_clf, artifact_path, signature=signature)

    print("Model saved in run %s" % run.info.run_id)


C:\Users\jreyarij\AppData\Local\Temp\ipykernel_23904\2415407220.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)
c:\Users\jreyarij\AppData\Local\anaconda3\envs\practiceEnv\lib\site-packages\xgboost\core.py:617: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)


Precisión:	 [0.82 0.95 0.84] 0.8679870326822101
Recall:		 [0.8  0.95 0.85] 0.8689275604084522


c:\Users\jreyarij\AppData\Local\anaconda3\envs\practiceEnv\lib\site-packages\mlflow\models\signature.py:144: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input)


Model saved in run a3ac12b08fa84c97b2e7c06cbe64a7aa


### Load model artifact and run prediction

In [11]:
run_id = "a3ac12b08fa84c97b2e7c06cbe64a7aa"
model_uri = "runs:/{}/model".format(run_id)
loaded_model = mlflow.sklearn.load_model(model_uri)
loaded_model.predict(X_test)

array([0, 1, 1, 2, 0, 0, 1, 2, 2, 2, 1, 2, 1, 0, 2, 2, 1, 1, 1, 2, 1, 0,
       1, 0, 1, 1, 0, 2, 1, 2, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2,
       1, 2, 2, 1, 1, 0, 1, 0, 1, 2, 2, 2, 0, 1, 2, 0, 0, 2, 0, 2, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 2, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1,
       1, 2, 1], dtype=int64)

# 2. Inspect MLflow UI

To inspect the model we just deployed, simply run ``mlflow ui``